# Mastodon analysis
Expected to be run in a notebook

DuckDB’s Python client can be used [directly in Jupyter notebook](https://duckdb.org/docs/guides/python/jupyter)

First step is import the relevant librariesSet and configure to directly output data to Pandas and to simplify the output that is printed to the notebook.



In [ ]:
import duckdb
import pandas as pd
import seaborn as sns

%load_ext sql
%sql duckdb:///:memory:

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Load [HTTPFS DuckDB extension](https://duckdb.org/docs/extensions/httpfs.html) for reading remote/writing remote files of object storage using the S3 API

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;

In [ ]:
%%sql
set s3_endpoint='localhost:9000';
set s3_access_key_id='minio';
set s3_secret_access_key='minio123';
set s3_use_ssl=false;
set s3_region='us-east-1';
set s3_url_style='path';

In [ ]:
%%sql
select *
from read_parquet('s3://mastodon/topics/mastodon-topic/partition=0/*');

## DuckDB SQL to process Mastodon activity
Run SQL
- cleanup any existing termporary tables
- create empty `language` lookup table and load languages from [language.csv](../duckdb/language.csv)
- create `mastodon_toot_raw` table by loading romote parquet files (from s3). Note the `created_at` timestamp is calculated as number of seconds from epoc
- final table `mastodon_toot` is a join of `mastodon_toot_raw` to `language`

In [ ]:
%%sql
drop table if exists mastodon_toot_raw;
drop table if exists mastodon_toot;
drop table if exists language;

CREATE TABLE language(lang_iso VARCHAR PRIMARY KEY, language_name VARCHAR);

insert into language
select *
from read_csv('../duckdb/language.csv', AUTO_DETECT=TRUE, header=True);

create table mastodon_toot_raw as
select m_id
, created_at, ('EPOCH'::TIMESTAMP + INTERVAL (created_at::INT) seconds)::TIMESTAMPTZ  as created_tz
, app
, url
, regexp_replace(regexp_replace(url, '^http[s]://', ''), '/.*$', '') as from_instance
, base_url
, language
, favourites
, username
, bot
, tags
, characters
, mastodon_text
FROM read_parquet('../data_tmp/all_toots.parquet');

create table mastodon_toot as
select mr.*, ln.language_name
from mastodon_toot_raw mr left outer join language ln on (mr.language = ln.lang_iso);

## Daily Mastodon usage

We can query the `mastodon_toot` table directly to see the number of _toots_, _users_ each day by counting and grouping the activity by the day

We can use the [mode](https://duckdb.org/docs/sql/aggregates.html#statistical-aggregates) aggregtae function to find the most frequent "bot" and "not-bot" users to find the most active Mastodon users



In [ ]:
%%sql
select strftime(created_tz, '%Y/%m/%d %a') as "Created day"
, count(*) as "Num toots"
, count(distinct(username)) as "Num users"
, count(distinct(from_instance)) as "Num urls"
, mode(case when bot='False' then username end) as "Most freq non-bot"
, mode(case when bot='True' then username end) as "Most freq bot"
, mode(base_url) as "Most freq host"
from mastodon_toot
where created_tz between TIMESTAMP '2023-02-07 13:00:00' and TIMESTAMP '2023-02-18 12:59:59' 
group by 1
order by 1
;

# The Mastodon app landscape
What clients are used to access mastodon instances

We take the query the `mastodon_toot` table, excluding "bots" and load query results into the `mastodon_app_df` Panda dataframe

In [ ]:
%%sql
mastodon_app_df << 
    select *
    from mastodon_toot
    where app is not null 
    and app <> ''
    and bot='False';

[Seaborn](https://seaborn.pydata.org/) is a visualization library for statistical graphics  in Python, built on the top of [matplotlib](https://matplotlib.org/). It also works really well with Panda data structures.


We can use [seaborn.countplot](https://seaborn.pydata.org/generated/seaborn.countplot.html) to show the counts of Mastodon app usage observations in each categorical bin using bars. Note, we are limiting this to the 10 highest occurances by specifying `mastodon_app_df.app.value_counts().iloc[:10]`





In [ ]:
sns.countplot(data=mastodon_app_df, y="app", order=mastodon_app_df.app.value_counts().iloc[:10].index)

# Time of day Mastodon usage
Let's see when Mastodon is used throughout the day and night. I want to get a raw hourly cound of _toots_ each hour of each day. We can load the results of this query into the `mastodon_usage_df` dataframe

In [ ]:
%%sql
mastodon_usage_df << 
    select strftime(created_tz, '%Y/%m/%d %a') as created_day
    , date_part('hour', created_tz) as created_hour
    , count(*) as num
    from mastodon_toot
    group by 1,2 
    order by 1,2;

In [ ]:
# sns.lineplot(data=mastodon_usage_df, x="created_hour", y="num")
df = mastodon_usage_df[mastodon_usage_df['created_day'].isin(['2023/02/04 Sat', '2023/02/05 Sun', '2023/02/11 Sat', '2023/02/12 Sun',  '2023/02/15 Wed',  '2023/02/16 Thu',  '2023/02/17 Fri'] ) ]
sns.lineplot(data=df  , x="created_hour", y="num", hue="created_day").set_xticks(range(24))

# Language usage
A wildly inaccurate investigation of language tags

In [ ]:
%%sql
mastodon_usage_df << 
    select *
    from mastodon_toot;

In [ ]:
sns.countplot(data=mastodon_usage_df, y="language_name", order=mastodon_usage_df.language_name.value_counts().iloc[:20].index)

In [ ]:
%%sql
mastodon_lang_df << 
select *
from mastodon_toot
where characters < 200
and language not in ('unknown');

In [ ]:
sns.boxplot(data=mastodon_lang_df, x="characters", y="language_name", whis=100, orient="h", order=mastodon_lang_df.language_name.value_counts().iloc[:20].index)


# Trending topics

# Random stuff

How frequently do _toots_ mention topical concepts such as the _superbowl_, _balloons_ or _ChatGPT_

In [ ]:
%%sql
select strftime(created_tz, '%Y/%m/%d %a') as "Created day"
, count(*) as "Num toots"
, sum(case when mastodon_text ilike '%balloon%' then 1 else 0 end) as cnt_balloon
, sum(case when mastodon_text ilike '%earthquake%' then 1 else 0 end) as cnt_earthquake
, sum(case when mastodon_text ilike '%superbowl%' then 1 else 0 end) as cnt_superbowl
, sum(case when mastodon_text ilike '%chatgpt%' then 1 else 0 end) as cnt_chatgpt
from mastodon_toot
where created_tz between TIMESTAMP '2023-02-07 13:00:00' and TIMESTAMP '2023-02-18 12:59:59' 
group by 1
order by 1
;